In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from IPython import display
import random
import copy
from itertools import product
from multiprocessing import Pool

In [ ]:
target = cv2.imread('/project/data/images/ilya.jpg')
target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)
target = target/255.
plt.imshow(target)

In [ ]:
class PolygonIndivid:
    
    def __init__(self, target_image, amount_of_shapes):
        self.target_image = target_image
        self.amount_of_shapes = amount_of_shapes
        
        self.image = self.init_image()
        self.points = self.generate_points(self.amount_of_shapes)
        self.colors = self.generate_colors(self.amount_of_shapes)
        
        self.draw(self.points, self.colors)
        
        
    def init_image(self):
        return np.zeros(self.target_image.shape)
        
        
    def generate_colors(self, amount):
        colors = np.random.random((amount, 4))
        return colors
    
    
    def generate_points(self, amount, cardinality=3):
        points = []
        for _ in range(amount):
            x = np.random.randint(0, self.image.shape[1], cardinality)
            y = np.random.randint(0, self.image.shape[0], cardinality)
            points.append(np.stack([x,y],axis=1))
        points = np.array(points)
        return points
    
    
    def draw(self, polygons, colors):
        for (polygon, color) in zip(polygons, colors):
            tmp_img = self.image.copy()
            tmp_img = cv2.fillPoly(tmp_img, [polygon], color[:3])
            alpha = color[3]
            self.image = cv2.addWeighted(tmp_img,alpha,self.image,1-alpha,0)

In [ ]:
class CircleIndivid:
    
    def __init__(self, target_image, amount_of_shapes, min_radius, max_radius):
        self.target_image = target_image
        self.amount_of_shapes = amount_of_shapes
        self.min_radius = min_radius
        self.max_radius = max_radius
        self.image = self.init_image()
        self.points = self.generate_points(self.amount_of_shapes)
        self.colors = self.generate_colors(self.amount_of_shapes)
        self.draw(self.points, self.colors)
        
        
    def init_image(self):
        return np.zeros(self.target_image.shape)
        
        
    def generate_colors(self, amount):
        colors = np.random.random((amount, 4))
        return colors
    
    
    def generate_points(self, amount, cardinality=1):
        points = []
        for _ in range(amount):
            x = np.random.randint(0, self.image.shape[1], cardinality)
            y = np.random.randint(0, self.image.shape[0], cardinality)
            points.append(np.stack([x,y],axis=1))
        points = np.array(points)
        return points
    
    
    def draw(self, points, colors):
        for (point, color) in zip(points, colors):
            tmp_img = self.image.copy()
            tmp_img = cv2.circle(tmp_img, tuple(point[0]), np.random.randint(self.min_radius,self.max_radius+1), tuple(color[:3]), -1)
            alpha = color[3]
            self.image = cv2.addWeighted(tmp_img,alpha,self.image,1-alpha,0)

In [ ]:
class Population:
    
    def __init__(self, target_image, best):
        self.target_image = target_image
        self.best = best
    
    
    def loss(self, individs):
        ind_imgs = np.array([i.image for i in individs])
        mse = (ind_imgs - self.target_image)**2/np.prod(self.target_image.shape)
        mse = np.sum(mse, axis=(1,2,3))
        return mse
    
    
    def mutate_ind(self, ind, min_amount, max_amount):
        amount = np.random.randint(min_amount, max_amount)
        points, colors = ind.generate_points(amount), ind.generate_colors(amount)
        ind.draw(points, colors)
        return ind
    
    
    def select_best(self, individs):
        mse = self.loss(individs)
        self.error = np.min(mse)
        sorted_inds = [x[1] for x in sorted(zip(mse, individs), key=lambda x: x[0])]
        return sorted_inds[0]

In [ ]:
def show_real_and_best(pop, history):
    history.append(pop.error)
    figure = plt.figure(figsize=(20,10))
    figure.add_subplot(2,2,1)
    plt.imshow(pop.target_image)
    figure.add_subplot(2,2,2)
    plt.imshow(pop.best.image)
    figure.add_subplot(2,2,(3,4))
    plt.plot(history)
    plt.show()

In [ ]:
def run(pop, children=10, min_amount=1, max_amount=10):
    children = [copy.deepcopy(pop.best) for _ in range(children)]
    children = [pop.mutate_ind(child, min_amount, max_amount) for child in children]
    new_best = pop.select_best(children)
    pop.best = new_best

In [ ]:
pop = Population(target, CircleIndivid(target, 0, 5,20))
history = []

In [ ]:
pop.best.min_radius = 2
pop.best.max_radius = 3

In [ ]:
history = []


EPOCHES = 1000

POP_SIZE = 30
MIN_NEW_SHAPES = 1
MAX_NEW_SHAPES = 3

for i in range(len(history),EPOCHES):
    run(pop, POP_SIZE, MIN_NEW_SHAPES, MAX_NEW_SHAPES)
    show_real_and_best(pop, history)
    #cv2.imwrite('/project/data/anime/{}.png'.format(i), (pop.best.image*255).astype(np.uint8))
    display.clear_output(wait=True)